# Text Classification:-

### Train a deep neural network for classification tickets(‘body’ column) data with respect to ‘urgency’.

### Setting Path

In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['all_tickets-1551435513304.csv']


In [18]:
from keras.models import Sequential

from keras.layers import Dense, LSTM, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D

from keras.preprocessing import sequence

### Reading Data

In [19]:
data = pd.read_csv("../input/all_tickets-1551435513304.csv",header=0)

In [20]:
data.head(10)

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4
5,mail,verification warning hi has got attached pleas...,1,4,3,7,89,3,4
6,mail,please dear looks blacklisted receiving mails ...,1,4,3,87,89,3,4
7,prod servers tunneling,prod tunneling va la tunneling la host si la s...,1,6,22,21,95,3,4
8,access request,dear modules report report cost thank much reg...,1,4,1,7,66,3,4
9,reset passwords for our client and,passwords client dear please passwords thank,1,5,2,76,4,3,4


### Getting Column Names

In [21]:
data.columns

Index(['title', 'body', 'ticket_type', 'category', 'sub_category1',
       'sub_category2', 'business_service', 'urgency', 'impact'],
      dtype='object')

### Dropping Irrelevant Columns

In [22]:
data.drop(index=1,columns=['title', 'ticket_type', 'category', 'sub_category1',
       'sub_category2', 'business_service', 'impact'],inplace = True)

In [23]:
data.head(5)

,body,urgency
0,hi since recruiter lead permission approve req...,3
2,work experience user hi work experience studen...,3
3,requesting meeting hi please help follow equip...,3
4,re expire days hi ask help update passwords co...,3
5,verification warning hi has got attached pleas...,3


### Text Cleaning

In [24]:
import re
from nltk.corpus import stopwords

data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data['body'] = data['body'].apply(clean_text)
data['body'] = data['body'].str.replace('\d+', '')

In [25]:
data.head(5)

,body,urgency
0,hi since recruiter lead permission approve req...,3
1,work eperience user hi work eperience student ...,3
2,requesting meeting hi please help follow equip...,3
3,epire days hi ask help update passwords collea...,3
4,verification warning hi got attached please ad...,3


### Tokenizing the Texts

In [26]:
from keras.preprocessing.text import Tokenizer

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['body'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 12149 unique tokens.


### Truncate and pad the input sequences so that they are all in the same length for modeling.

In [27]:
from keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(data['body'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (48548, 1000)


In [29]:
print('Shape of label tensor:', Y.shape)
print(type(Y))
print(Y)

Shape of label tensor: (48548, 4)
<class 'numpy.ndarray'>
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 ...
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]]


### Splitting the data inti Train and Test

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(38838, 1000) (38838, 4)
(9710, 1000) (9710, 4)


### Declaring the model and fitting the data onto the model:-

In [15]:
from keras.models import Sequential
from keras.layers import SpatialDropout1D
from keras.callbacks import *
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 50
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 34954 samples, validate on 3884 samples
Epoch 1/50
34954/34954 [==============================] - 901s 26ms/step - loss: 0.3992 - acc: 0.8240 - val_loss: 0.2923 - val_acc: 0.8587
Epoch 2/50
34954/34954 [==============================] - 891s 25ms/step - loss: 0.3023 - acc: 0.8544 - val_loss: 0.2914 - val_acc: 0.8597
Epoch 3/50
34954/34954 [==============================] - 883s 25ms/step - loss: 0.2804 - acc: 0.8666 - val_loss: 0.2860 - val_acc: 0.8653
Epoch 4/50
34954/34954 [==============================] - 880s 25ms/step - loss: 0.2594 - acc: 0.8787 - val_loss: 0.2841 - val_acc: 0.8705
Epoch 5/50
34954/34954 [==============================] - 880s 25ms/step - loss: 0.2347 - acc: 0.8936 - val_loss: 0.2818 - val_acc: 0.8708
Epoch 6/50
34954/3495

### Test Accuracy

In [16]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

9710/9710 [==============================] - 154s 16ms/step
Test set
  Loss: 0.317
  Accuracy: 0.868
